In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, make_scorer
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import joblib
import os

In [2]:
df = pd.read_csv('DF_FINAL.csv')
df['Week_End_Date'] = pd.to_datetime(df['Week_End_Date'])
df

,Territory_code,Plant_code,Brand_code,Package,Week_End_Date,Sum of Plan,fact,row_id
0,A,a,AC,TP 0.2,2023-01-06,1.000000,0.010000,A_a_AC_TP 0.2
1,A,a,AC,TP 0.2,2023-01-13,1.000000,4.010000,A_a_AC_TP 0.2
2,A,a,AC,TP 0.2,2023-01-20,61.000000,6.010000,A_a_AC_TP 0.2
3,A,a,AC,TP 0.2,2023-01-27,0.000000,8.010000,A_a_AC_TP 0.2
4,A,a,AC,TP 0.2,2023-02-03,18.879000,9.520000,A_a_AC_TP 0.2
...,...,...,...,...,...,...,...,...
58305,I,an,AD,PET 1.0,2024-11-29,54.766687,78.192999,I_an_AD_PET 1.0
58306,I,an,AD,PET 1.0,2024-12-06,65.133425,171.193998,I_an_AD_PET 1.0
58307,I,an,AD,PET 1.0,2024-12-13,59.458602,46.503001,I_an_AD_PET 1.0
58308,I,an,AD,PET 1.0,2024-12-20,77.697035,8.463000,I_an_AD_PET 1.0


In [3]:
EPSILON = 1e-5  # Для избежания деления на 0

In [4]:
def create_features(df_row: pd.DataFrame) -> pd.DataFrame:
    df_row = df_row.sort_values('Week_End_Date').copy()

    # Временные признаки
    df_row['week_number'] = df_row['Week_End_Date'].dt.isocalendar().week
    df_row['month'] = df_row['Week_End_Date'].dt.month
    df_row['quarter'] = df_row['Week_End_Date'].dt.quarter

    # Лаги fact
    for lag in [1, 2, 3, 4, 5, 7]:
        df_row[f'lag_{lag}'] = df_row['fact'].shift(lag)

    # Скользящие средние и std
    for window in [3, 5, 7]:
        df_row[f'rolling_mean_{window}'] = df_row['fact'].rolling(window).mean()
        df_row[f'rolling_std_{window}'] = df_row['fact'].rolling(window).std()

    # Трендовые признаки
    df_row['trend_3'] = df_row['fact'] - df_row['fact'].shift(3)
    df_row['momentum'] = (df_row['fact'] - df_row['fact'].shift(1)) / \
                         (df_row['fact'].shift(2) - df_row['fact'].shift(3) + 1e-5)

    # Удаление строк с NaN после лагов и rolling
    df_row.dropna(inplace=True)

    return df_row

In [5]:
# 2. Применяем функцию
features_df = df.groupby('row_id').apply(create_features).reset_index(drop=True)

# 3. Проверяем результат
features_df

C:\Users\Anton\AppData\Local\Temp\ipykernel_26280\2806616002.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  features_df = df.groupby('row_id').apply(create_features).reset_index(drop=True)


,Territory_code,Plant_code,Brand_code,Package,Week_End_Date,Sum of Plan,fact,row_id,week_number,month,...,lag_5,lag_7,rolling_mean_3,rolling_std_3,rolling_mean_5,rolling_std_5,rolling_mean_7,rolling_std_7,trend_3,momentum
0,A,a,AC,TP 0.2,2023-02-24,9.123000,47.560000,A_a_AC_TP 0.2,8,2,...,6.010000,0.010,71.970332,55.097398,46.688199,52.121213,34.780142,47.169933,38.040000,0.113633
1,A,a,AC,TP 0.2,2023-03-03,70.298003,2.863000,A_a_AC_TP 0.2,9,3,...,8.010000,4.010,27.906000,22.830603,45.658799,53.117394,34.616285,47.296458,-132.192996,0.439235
2,A,a,AC,TP 0.2,2023-03-10,0.165000,1.912000,A_a_AC_TP 0.2,10,3,...,9.520000,6.010,17.445000,26.084690,44.137199,54.502347,34.030856,47.732905,-31.382999,-0.066667
3,A,a,AC,TP 0.2,2023-03-17,0.196000,1.912000,A_a_AC_TP 0.2,11,3,...,135.055995,8.010,2.229000,0.549060,17.508400,21.524977,33.159714,48.338744,-45.648000,-0.000000
4,A,a,AC,TP 0.2,2023-03-24,0.196000,1.912000,A_a_AC_TP 0.2,12,3,...,33.294999,9.520,1.912000,0.000000,11.231800,20.312256,32.072856,49.039303,-0.951000,-0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54091,I,an,AD,PET 1.0,2024-11-29,54.766687,78.192999,I_an_AD_PET 1.0,48,11,...,2.123000,0.010,53.542334,46.408603,32.129400,44.006732,23.254286,39.002214,78.182999,-423100.376129
54092,I,an,AD,PET 1.0,2024-12-06,65.133425,171.193998,I_an_AD_PET 1.0,49,12,...,0.010000,0.010,110.603666,52.515394,66.366200,71.050834,47.709143,66.189900,171.183998,1.128461
54093,I,an,AD,PET 1.0,2024-12-13,59.458602,46.503001,I_an_AD_PET 1.0,50,12,...,0.010000,2.123,98.629999,64.809061,75.664800,62.753417,54.049143,63.151414,-35.921001,29.470853
54094,I,an,AD,PET 1.0,2024-12-20,77.697035,8.463000,I_an_AD_PET 1.0,51,12,...,0.010000,0.010,75.386666,85.123698,77.355400,60.270449,55.256714,62.016485,-69.729999,-0.409028


In [6]:
def split_train_test(features_df, n_test=4):
    train_list = []
    test_list = []
    # Группируем по уникальному ряду
    for row_id, group in features_df.groupby('row_id'):
        group = group.sort_values('Week_End_Date')
        # Тест — последние n_test строк, трейн — всё остальное
        test_part = group.tail(n_test)
        train_part = group.iloc[:-n_test]
        test_list.append(test_part)
        train_list.append(train_part)
    # Собираем обратно в датафреймы
    train_df = pd.concat(train_list).reset_index(drop=True)
    test_df = pd.concat(test_list).reset_index(drop=True)
    return train_df, test_df

In [7]:
# Применяем
train_df, test_df = split_train_test(features_df, n_test=4)

# Сохраняем
#train_df.to_csv('train.csv', index=False)
#test_df.to_csv('test.csv', index=False)

In [16]:
def process_single_row(row_id: str) -> dict:
    try:
        df_row = df[df['row_id'] == row_id].copy()
        df_feat = create_features(df_row)

        if len(df_feat) < 10:
            return {'row_id': row_id, 'status': 'too_short'}

        # Train / Test
        df_train = df_feat.iloc[:-4]
        df_test = df_feat.iloc[-4:]

        X_train = df_train.drop(columns=['fact', 'Week_End_Date', 'row_id',
                                         'Territory_code', 'Plant_code', 'Brand_code', 'Package', 'Sum of Plan'], errors='ignore')
        y_train = df_train['fact']
        X_test = df_test[X_train.columns]
        y_test = df_test['fact']

        # Grid Search
        model = XGBRegressor(objective='reg:squarederror', random_state=42)
        param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.05, 0.1],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0],
        }

        tscv = TimeSeriesSplit(n_splits=3)
        grid = GridSearchCV(model, param_grid, cv=tscv, scoring='neg_mean_absolute_error', n_jobs=-1)
        grid.fit(X_train, y_train)

        best_model = grid.best_estimator_
        # Создаём папку для моделей, если нужно
        os.makedirs('models', exist_ok=True)
        model_path = f'models/model_{row_id}.joblib'
        joblib.dump(best_model, model_path)
        
        
        y_pred = best_model.predict(X_test)

        # Обрезка отрицательных прогнозов
        y_pred = np.maximum(y_pred, 0)

        # Метрики
        mae = mean_absolute_error(y_test, y_pred)
        wape = np.sum(np.abs(y_test - y_pred)) / (np.sum(np.abs(y_test)) + 1e-5) * 100
        smape = 100 * np.mean(np.abs(y_pred - y_test) / ((np.abs(y_test) + np.abs(y_pred)) / 2 + 1e-5))

        return {
            'row_id': row_id,
            'MAE': mae,
            'WAPE': wape,
            'SMAPE': smape,
            'params': grid.best_params_,
            'status': 'ok'
        }

    except Exception as e:
        return {'row_id': row_id, 'status': f'error: {e}'}

In [17]:
# Получаем все уникальные ряды
unique_row_ids = df['row_id'].unique()

# Запускаем обработку
results = Parallel(n_jobs=-1, verbose=10)(
    delayed(process_single_row)(row_id) for row_id in unique_row_ids
)

# Собираем в DataFrame
metrics_df = pd.DataFrame(results)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   17.1s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   27.7s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:   33.3s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:   44.4s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   56.5s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

In [18]:
metrics_df

,row_id,MAE,WAPE,SMAPE,params,status
0,A_a_AC_TP 0.2,0.567235,29.667062,67.886600,"{'colsample_bytree': 0.8, 'learning_rate': 0.1...",ok
1,A_a_AC_TP 1.0,34.818678,67.237642,94.743511,"{'colsample_bytree': 0.8, 'learning_rate': 0.0...",ok
2,A_a_AD_PET 1.0,38.552308,42.173769,86.844804,"{'colsample_bytree': 0.8, 'learning_rate': 0.0...",ok
3,A_b_AA_PET 0.5,14.092477,15.596096,72.397928,"{'colsample_bytree': 1.0, 'learning_rate': 0.1...",ok
4,A_b_AA_PET 1.0,11.760786,38.366233,117.012435,"{'colsample_bytree': 1.0, 'learning_rate': 0.1...",ok
...,...,...,...,...,...,...
597,I_am_AH_PET 1.0,85.233525,31.633612,51.567827,"{'colsample_bytree': 1.0, 'learning_rate': 0.1...",ok
598,I_am_AJ_PET 1.0,32.197129,36.491545,56.525742,"{'colsample_bytree': 1.0, 'learning_rate': 0.0...",ok
599,I_an_AC_PET 0.3,1.831529,20.611394,104.516797,"{'colsample_bytree': 1.0, 'learning_rate': 0.1...",ok
600,I_an_AC_TP 2.0,138.470680,51.588519,110.785929,"{'colsample_bytree': 0.8, 'learning_rate': 0.1...",ok


In [19]:
metrics_df['Wape_cliped'] = metrics_df['WAPE'].clip(upper=300)
metrics_df[['MAE', 'WAPE', 'SMAPE', 'Wape_cliped']].describe()

,MAE,WAPE,SMAPE,Wape_cliped
count,602.000000,602.000000,602.000000,602.000000
mean,75.128427,53.688279,67.068041,42.466033
std,251.555252,163.182392,45.134674,47.520634
min,0.000568,0.105515,1.052081,0.105515
25%,2.640271,17.705339,31.140332,17.705339
50%,14.163472,29.706610,59.001876,29.706610
75%,46.601736,47.730084,95.201105,47.730084
max,4550.873478,2705.296474,199.523512,300.000000


In [20]:
zzzz = train_df.drop(columns=['fact', 'Week_End_Date', 'row_id',
                                         'Territory_code', 'Plant_code', 'Brand_code', 'Package', 'Sum of Plan'], errors='ignore')
zzzz

,week_number,month,quarter,lag_1,lag_2,lag_3,lag_4,lag_5,lag_7,rolling_mean_3,rolling_std_3,rolling_mean_5,rolling_std_5,rolling_mean_7,rolling_std_7,trend_3,momentum
0,8,2,1,33.294999,135.055995,9.520000,8.010000,6.010000,0.010,71.970332,55.097398,46.688199,52.121213,34.780142,47.169933,38.040000,1.136327e-01
1,9,3,1,47.560000,33.294999,135.055995,9.520000,8.010000,4.010,27.906000,22.830603,45.658799,53.117394,34.616285,47.296458,-132.192996,4.392351e-01
2,10,3,1,2.863000,47.560000,33.294999,135.055995,9.520000,6.010,17.445000,26.084690,44.137199,54.502347,34.030856,47.732905,-31.382999,-6.666661e-02
3,11,3,1,1.912000,2.863000,47.560000,33.294999,135.055995,8.010,2.229000,0.549060,17.508400,21.524977,33.159714,48.338744,-45.648000,-0.000000e+00
4,12,3,1,1.912000,1.912000,2.863000,47.560000,33.294999,9.520,1.912000,0.000000,11.231800,20.312256,32.072856,49.039303,-0.951000,-0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51683,44,11,4,2.123000,0.010000,0.010000,2.123000,0.010000,0.010,0.714333,1.219941,0.855200,1.157338,0.613714,1.031038,0.000000,-2.113000e+05
51684,45,11,4,0.010000,2.123000,0.010000,0.010000,2.123000,0.010,0.714333,1.219941,0.432600,0.944962,0.613714,1.031038,0.000000,0.000000e+00
51685,46,11,4,0.010000,0.010000,2.123000,0.010000,0.010000,0.010,0.010000,0.000000,0.432600,0.944962,0.613714,1.031038,-2.113000,-0.000000e+00
51686,47,11,4,0.010000,0.010000,0.010000,2.123000,0.010000,2.123,27.481334,47.581746,16.915400,36.631850,12.085286,31.026453,82.414002,8.241400e+06


In [9]:
train_df

,Territory_code,Plant_code,Brand_code,Package,Week_End_Date,Sum of Plan,fact,row_id,week_number,month,...,lag_5,lag_7,rolling_mean_3,rolling_std_3,rolling_mean_5,rolling_std_5,rolling_mean_7,rolling_std_7,trend_3,momentum
0,A,a,AC,TP 0.2,2023-02-24,9.123000,47.560000,A_a_AC_TP 0.2,8,2,...,6.010000,0.010,71.970332,55.097398,46.688199,52.121213,34.780142,47.169933,38.040000,1.136327e-01
1,A,a,AC,TP 0.2,2023-03-03,70.298003,2.863000,A_a_AC_TP 0.2,9,3,...,8.010000,4.010,27.906000,22.830603,45.658799,53.117394,34.616285,47.296458,-132.192996,4.392351e-01
2,A,a,AC,TP 0.2,2023-03-10,0.165000,1.912000,A_a_AC_TP 0.2,10,3,...,9.520000,6.010,17.445000,26.084690,44.137199,54.502347,34.030856,47.732905,-31.382999,-6.666661e-02
3,A,a,AC,TP 0.2,2023-03-17,0.196000,1.912000,A_a_AC_TP 0.2,11,3,...,135.055995,8.010,2.229000,0.549060,17.508400,21.524977,33.159714,48.338744,-45.648000,-0.000000e+00
4,A,a,AC,TP 0.2,2023-03-24,0.196000,1.912000,A_a_AC_TP 0.2,12,3,...,33.294999,9.520,1.912000,0.000000,11.231800,20.312256,32.072856,49.039303,-0.951000,-0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51683,I,an,AD,PET 1.0,2024-11-01,171.247923,0.010000,I_an_AD_PET 1.0,44,11,...,0.010000,0.010,0.714333,1.219941,0.855200,1.157338,0.613714,1.031038,0.000000,-2.113000e+05
51684,I,an,AD,PET 1.0,2024-11-08,163.008797,0.010000,I_an_AD_PET 1.0,45,11,...,2.123000,0.010,0.714333,1.219941,0.432600,0.944962,0.613714,1.031038,0.000000,0.000000e+00
51685,I,an,AD,PET 1.0,2024-11-15,163.008797,0.010000,I_an_AD_PET 1.0,46,11,...,0.010000,0.010,0.010000,0.000000,0.432600,0.944962,0.613714,1.031038,-2.113000,-0.000000e+00
51686,I,an,AD,PET 1.0,2024-11-22,163.008797,82.424002,I_an_AD_PET 1.0,47,11,...,0.010000,2.123,27.481334,47.581746,16.915400,36.631850,12.085286,31.026453,82.414002,8.241400e+06


In [8]:
test_df

,Territory_code,Plant_code,Brand_code,Package,Week_End_Date,Sum of Plan,fact,row_id,week_number,month,...,lag_5,lag_7,rolling_mean_3,rolling_std_3,rolling_mean_5,rolling_std_5,rolling_mean_7,rolling_std_7,trend_3,momentum
0,A,a,AC,TP 0.2,2024-12-06,0.000000,0.010000,A_a_AC_TP 0.2,49,12,...,4.765000,5.716000,1.912000,1.647180,2.102200,1.562652,2.183714,1.879221,-0.951000,-1.499992
1,A,a,AC,TP 0.2,2024-12-13,12.501315,2.863000,A_a_AC_TP 0.2,50,12,...,3.814000,0.010000,1.912000,1.647180,1.912000,1.344917,2.591286,1.620824,0.000000,285299.992561
2,A,a,AC,TP 0.2,2024-12-20,12.501315,3.814000,A_a_AC_TP 0.2,51,12,...,0.961000,4.765000,2.229000,1.979664,2.482600,1.442263,2.455429,1.437777,0.951000,-0.333335
3,A,a,AC,TP 0.2,2024-12-27,7.500789,0.961000,A_a_AC_TP 0.2,52,12,...,2.863000,3.814000,2.546000,1.452676,2.102200,1.562652,2.047857,1.392121,0.951000,-0.999996
4,A,a,AC,TP 1.0,2024-12-06,0.000000,0.010000,A_a_AC_TP 1.0,49,12,...,0.010000,23.252999,105.674334,123.153775,63.408600,104.560564,55.256714,88.812530,0.000000,-0.315777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2403,I,an,AC,TP 2.0,2024-12-27,60.227004,4.236000,I_an_AC_TP 2.0,52,12,...,0.010000,4.236000,105.680336,179.379013,214.732999,331.576569,153.383571,290.298051,-752.377989,-0.009522
2404,I,an,AD,PET 1.0,2024-12-06,65.133425,171.193998,I_an_AD_PET 1.0,49,12,...,0.010000,0.010000,110.603666,52.515394,66.366200,71.050834,47.709143,66.189900,171.183998,1.128461
2405,I,an,AD,PET 1.0,2024-12-13,59.458602,46.503001,I_an_AD_PET 1.0,50,12,...,0.010000,2.123000,98.629999,64.809061,75.664800,62.753417,54.049143,63.151414,-35.921001,29.470853
2406,I,an,AD,PET 1.0,2024-12-20,77.697035,8.463000,I_an_AD_PET 1.0,51,12,...,0.010000,0.010000,75.386666,85.123698,77.355400,60.270449,55.256714,62.016485,-69.729999,-0.409028


In [22]:
row_id = 'E_u_AD_PET 0.5'
model_path = f'models/model_{row_id}.joblib'
model = joblib.load(model_path)

In [23]:
test_row = test_df[test_df['row_id'] == row_id].copy()
test_row = test_row.sort_values('Week_End_Date')

In [24]:
feature_cols = [
    'week_number', 'month', 'quarter', 'lag_1', 'lag_2', 'lag_3', 'lag_4',
    'lag_5', 'lag_7', 'rolling_mean_3', 'rolling_std_3', 'rolling_mean_5',
    'rolling_std_5', 'rolling_mean_7', 'rolling_std_7', 'trend_3',
    'momentum'
]
X_test = test_row[feature_cols]
y_true = test_row['fact'].values

In [25]:
from scipy.stats import t

In [26]:
y_pred = model.predict(X_test)

wapes = np.abs(y_true - y_pred) / np.abs(y_true) * 100

mean_wape = np.mean(wapes)
std_wape = np.std(wapes, ddof=1)
n = len(wapes)
t_crit = t.ppf(0.975, df=n-1)
ci_low = mean_wape - t_crit * std_wape / np.sqrt(n)
ci_high = mean_wape + t_crit * std_wape / np.sqrt(n)

print('WAPE по неделям:', wapes)
print(f"Mean WAPE: {mean_wape:.2f}%")
print(f"95% CI: [{ci_low:.2f}%; {ci_high:.2f}%]")

WAPE по неделям: [15.54841075 79.41194096 14.28661143 18.0638033 ]
Mean WAPE: 31.83%
95% CI: [-18.71%; 82.37%]


In [27]:
y_pred

array([ 49.974228,  60.675323, 111.1083  ,  95.2369  ], dtype=float32)

In [29]:
y_true

array([ 59.17499996,  33.81899954,  97.21899868, 116.233001  ])

In [30]:
row_ids_CI = [
    'E_u_AD_PET 0.5',
    'I_am_AE_PET 0.45',
    'A_c_AJ_PET 0.5',
    'D_o_AC_TP 0.2',
    'I_am_AA_PET 0.5'
]

In [60]:
results_CI = []
for row_id in row_ids_CI:
    # Загрузка модели
    model_path = f'models/model_{row_id}.joblib'
    model = joblib.load(model_path)

    # Получение строк по ряду
    test_row = test_df[test_df['row_id'] == row_id].copy().sort_values('Week_End_Date')

    feature_cols = [
    'week_number', 'month', 'quarter', 'lag_1', 'lag_2', 'lag_3', 'lag_4',
    'lag_5', 'lag_7', 'rolling_mean_3', 'rolling_std_3', 'rolling_mean_5',
    'rolling_std_5', 'rolling_mean_7', 'rolling_std_7', 'trend_3',
    'momentum']
    X_test = test_row[feature_cols]
    y_true = test_row['fact'].values
    y_pred = model.predict(X_test)

    # WAPE по неделям
    wapes = np.abs(y_true - y_pred) / np.abs(y_true) * 100
    # Обрезаем выбросы (wape > 60%)
    wapes_no_outliers = wapes[wapes <= 80]
    
    mean_wape = np.mean(wapes_no_outliers)
    
    if len(wapes_no_outliers) > 1:
        std_wape = np.std(wapes_no_outliers, ddof=1)
        n = len(wapes_no_outliers)
        t_crit = t.ppf(0.975, df=n-1)
        ci_low = mean_wape - t_crit * std_wape / np.sqrt(n)
        ci_high = mean_wape + t_crit * std_wape / np.sqrt(n)
        ci_low = max(0, ci_low)
        ci_high = min(300, ci_high)
    else:
        ci_low = None
        ci_high = None

    results_CI.append({
        'row_id': row_id,
        'Mean WAPE': round(mean_wape, 2),
        '95% CI lower': round(ci_low, 2),
        '95% CI upper': round(ci_high, 2),
        'WAPE по неделям': np.round(wapes, 2)
    })

In [61]:
df_results_CI = pd.DataFrame(results_CI)
df_results_CI

,row_id,Mean WAPE,95% CI lower,95% CI upper,WAPE по неделям
0,E_u_AD_PET 0.5,31.83,0,82.37,"[15.55, 79.41, 14.29, 18.06]"
1,I_am_AE_PET 0.45,11.79,0,42.71,"[26.13, 5.51, 3.73, 87316.42]"
2,A_c_AJ_PET 0.5,40.47,0,125.80,"[0.99, 63.56, 56.85, 548.84]"
3,D_o_AC_TP 0.2,16.31,0,59.26,"[36.27, 6.75, 98.17, 5.9]"
4,I_am_AA_PET 0.5,32.69,0,112.20,"[129.74, 26.43, 38.94, 30660.52]"
